In [32]:
import re
import os
import pandas as pd
import csv


def file_convert (txt_path,pid):
    
    # initialize dict, columns "pID", "interview_text" and tag
    csv_content = {}
    csv_content ["pID"] = pid
    csv_content ["interview_text"]= ""
    tag = -1


    with open(txt_path, "r", encoding="utf-8",errors="replace") as f:
        full_contents=[l.strip() for l in f.readlines()]
        for line in full_contents:
            # if "start of chat interview" in line, record interview text
            if tag == 0 and "Questionnaire" not in line:
                csv_content ["interview_text"]+=line+"\n"
            # if tag n detected, split and record content
            if tag > 0 and "Questionnaire" not in line:
                infos = line.split(":")
                if infos[0] != "":
                    csv_content[f"Q{tag}_"+str(infos[0])]=infos[-1]
            # if "Questionnaire" in line, set tag = n from (Qn)
            if "Questionnaire" in line:
                match = re.search(r"Questionnaire\s+(\d+)", line)
                tag = int(match.group(1))
            # if "start of chat interview" in line, set tag = 0
            if "Start of Chat Interview" in line:
                tag = 0
    
    return csv_content


TRANSCRIPTS_DIR = "../data/transcripts"
OUTPUT_DIR = "../data\\result_csvs"
file_names = [fname for fname in os.listdir(TRANSCRIPTS_DIR) if fname.endswith(".txt")]


for file_name in file_names:
    txt_path = os.path.join(TRANSCRIPTS_DIR, file_name)
    pid = file_name.split(".")[0]
    output = file_convert(txt_path,pid)
    with open(os.path.join(OUTPUT_DIR, pid+".csv"), "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=output.keys())
        writer.writeheader()
        writer.writerow(output)

